# Capstone Project - The Battle of Neighborhoods - Week 2

Introduction: COFFEE ETHNIC IN JEDDAH

In my local home city "Jeddah", Saudi Arabia has large and rich of coffee culture, the project is to try determine best location to start new business.

In [30]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [31]:
CLIENT_ID = 'IMQCHXD2HFOY4N42EFP5FJ3P51URBL1OWPJQT3CVHKHFQ0GJ' # your Foursquare ID
CLIENT_SECRET = '2OQWVXH01BHG0K0SYQEX5SKBBLXP5A2MT3MN3P5DANNQQAMU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IMQCHXD2HFOY4N42EFP5FJ3P51URBL1OWPJQT3CVHKHFQ0GJ
CLIENT_SECRET:2OQWVXH01BHG0K0SYQEX5SKBBLXP5A2MT3MN3P5DANNQQAMU


In [5]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Jeddah",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [6]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [8]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Jeddah', 'Jeddah')

In [9]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 21.548160599408067, 'lng': 39.2054861427913},
  'sw': {'lat': 21.533384517160297, 'lng': 39.18902370748613}},
 11)

In [10]:
d["geocode"]

{'what': '',
 'where': 'jeddah',
 'center': {'lat': 21.54238, 'lng': 39.19797},
 'displayString': 'Jeddah, Saudi Arabia',
 'cc': 'SA',
 'geometry': {'bounds': {'ne': {'lat': 21.817041, 'lng': 39.303589},
   'sw': {'lat': 21.314672, 'lng': 39.047565}}},
 'slug': 'jeddah',
 'longId': '72057594038033279'}

In [11]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [13]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [14]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 11


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '58449afcd25ded6f32671e7d',
  'name': 'Bon Cafe',
  'location': {'address': 'Macrona St، Aziziyah',
   'lat': 21.546609,
   'lng': 39.19562,
   'labeledLatLngs': [{'label': 'display', 'lat': 21.546609, 'lng': 39.19562}],
   'cc': 'SA',
   'neighborhood': 'Al-Azizeyyah',
   'city': 'جدة',
   'state': 'منطقة مكة',
   'country': 'المملكة العربية السعودية',
   'formattedAddress': ['Macrona St، Aziziyah',
    'جدة',
    'المملكة العربية السعودية']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-58449afcd25ded6f32671e7d-0'}

In [17]:
items [1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d94c741f3b98d0009009205',
  'name': 'Dunkin Dounat',
  'location': {'lat': 21.546753,
   'lng': 39.189772,
   'labeledLatLngs': [{'label': 'display',
     'lat': 21.546753,
     'lng': 39.189772}],
   'postalCode': '23333',
   'cc': 'SA',
   'city': 'جدة',
   'state': 'منطقة مكة',
   'country': 'المملكة العربية السعودية',
   'formattedAddress': ['جدة 23333', 'المملكة العربية السعودية']},
  'categories': [{'id': '4bf58dd8d48988d148941735',
    'name': 'Donut Shop',
    'pluralName': 'Donut Shops',
    'shortName': 'Donuts',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/donuts_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-5d94c741f3b98d0009009205-1'}

In [18]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': 'Macrona St، Aziziyah', 'lat': 21.546609, 'lng': 39.19562, 'labeledLatLngs': [{'label': 'display', 'lat': 21.546609, 'lng': 39.19562}], 'cc': 'SA', 'neighborhood': 'Al-Azizeyyah', 'city': 'جدة', 'state': 'منطقة مكة', 'country': 'المملكة العربية السعودية', 'formattedAddress': ['Macrona St، Aziziyah', 'جدة', 'المملكة العربية السعودية']}
{'lat': 21.546753, 'lng': 39.189772, 'labeledLatLngs': [{'label': 'display', 'lat': 21.546753, 'lng': 39.189772}], 'postalCode': '23333', 'cc': 'SA', 'city': 'جدة', 'state': 'منطقة مكة', 'country': 'المملكة العربية السعودية', 'formattedAddress': ['جدة 23333', 'المملكة العربية السعودية']}
{'lat': 21.542382080895436, 'lng': 39.19132500616264, 'labeledLatLngs': [{'label': 'display', 'lat': 21.542382080895436, 'lng': 39.19132500616264}], 'cc': 'SA', 'country': 'المملكة العربية السعودية', 'formattedAddress': ['المملكة العربية السعودية']}
{'address': 'alTahlia st.', 'crossStreet': 'Prince Sultan road', 'lat': 21.542704366263123, 'lng': 39.1952646096

,uid,name,shortname,address,postalcode,lat,lng
0,5d94c741f3b98d0009009205,Dunkin Dounat,Donuts,,23333,21.546753,39.189772
1,5dbc432d50db0f00082e8da5,Dankin Donuts,Coffee Shop,,23333,21.546191,39.189859


In [19]:
Jeddah_center = d["geocode"]["center"]
Jeddah_center

{'lat': 21.54238, 'lng': 39.19797}

In [24]:
from folium import plugins
lat = 21.54238
lng = 39.19797

map_Jeddah = folium.Map(location=[lat, lng], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Jeddah)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Jeddah.add_child(plugins.HeatMap(hm_data))

map_Jeddah

In [28]:
map_Jeddah = folium.Map(location=[lat, lng], zoom_start=16)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Jeddah)
map_Jeddah

The blue spot show the best location to open the new coffee shop